In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/studentcnn/pytorch/default/1/model (4).pth


In [2]:
!pip install detectors -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.8/616.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.2/126.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 5.2 MB/s eta 0:00:00


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F


In [4]:
import torch,timm,detectors
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.nn import KLDivLoss
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                         std=[0.2023, 0.1994, 0.2010])
])
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=128 , shuffle=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
teacher_model = timm.create_model("resnet34_cifar10", pretrained=True).to(device)


100%|██████████| 170M/170M [00:05<00:00, 33.8MB/s]
Downloading: "https://huggingface.co/edadaltocg/resnet34_cifar10/resolve/main/pytorch_model.bin" to /root/.cache/torch/hub/checkpoints/resnet34_cifar10.pth
100%|██████████| 81.3M/81.3M [00:00<00:00, 367MB/s]


In [5]:
# Define the StudentCNN model
class StudentCNN(nn.Module):
    def __init__(self):
        super(StudentCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)  # Adjusted for CIFAR-10
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # 32x32 -> 16x16
        x = self.pool(F.relu(self.conv2(x)))  # 16x16 -> 8x8
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

In [6]:
def train_with_kl_and_ce(student_model, teacher_model, loader, optimizer, epochs=150, temperature=2, alpha=0.7):
    kl_loss = nn.KLDivLoss(reduction='batchmean')
    ce_loss = nn.CrossEntropyLoss()
    student_model.train()
    teacher_model.eval()
    for epoch in range(epochs):
        total_loss = 0
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)

            with torch.no_grad():
                teacher_logits = teacher_model(images)
                teacher_probs = F.softmax(teacher_logits / temperature, dim=1)
              
            optimizer.zero_grad()
            student_logits = student_model(images)

            # KL Divergence Loss
            student_log_probs = F.log_softmax(student_logits / temperature, dim=1)

            loss_kl = kl_loss(student_log_probs, teacher_probs) * (temperature ** 2)

            # Cross-Entropy Loss
            loss_ce = ce_loss(student_logits, labels)

            # Combined Loss
            loss = alpha * loss_kl + (1 - alpha) * loss_ce

            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1}, Total Loss: {total_loss:.4f}")


In [7]:
student_model_kl = StudentCNN()

In [8]:


student_model_kl.to(device)
optimizer_kl =torch.optim.Adam(student_model_kl.parameters(), lr=1e-3, weight_decay=5e-4)



train_with_kl_and_ce(student_model_kl, teacher_model, train_loader, optimizer_kl)

Epoch 1, Total Loss: 1488.0335
Epoch 2, Total Loss: 1055.1606
Epoch 3, Total Loss: 885.0075
Epoch 4, Total Loss: 755.5135
Epoch 5, Total Loss: 635.7556
Epoch 6, Total Loss: 525.3563
Epoch 7, Total Loss: 423.0709
Epoch 8, Total Loss: 329.3782
Epoch 9, Total Loss: 254.6904
Epoch 10, Total Loss: 202.8731
Epoch 11, Total Loss: 171.9604
Epoch 12, Total Loss: 150.1092
Epoch 13, Total Loss: 145.5740
Epoch 14, Total Loss: 151.9224
Epoch 15, Total Loss: 133.8932
Epoch 16, Total Loss: 120.6918
Epoch 17, Total Loss: 125.5163
Epoch 18, Total Loss: 144.1575
Epoch 19, Total Loss: 122.1172
Epoch 20, Total Loss: 86.3089
Epoch 21, Total Loss: 88.5170
Epoch 22, Total Loss: 188.5788
Epoch 23, Total Loss: 134.7222
Epoch 24, Total Loss: 75.4949
Epoch 25, Total Loss: 52.0783
Epoch 26, Total Loss: 74.0434
Epoch 27, Total Loss: 285.6419
Epoch 28, Total Loss: 114.3109
Epoch 29, Total Loss: 60.4906
Epoch 30, Total Loss: 43.7149
Epoch 31, Total Loss: 46.5871
Epoch 32, Total Loss: 261.0933
Epoch 33, Total Loss: 1

In [9]:
torch.save(student_model_kl, '/kaggle/working/model.pth')

In [10]:
correct = 0
total = 0
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=128 , shuffle=False)
with torch.no_grad():
    for images, labels in test_loader:
        outputs = student_model_kl(images.to(device))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.to(device)).sum().item()

accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy:.2f}%')

Test Accuracy: 74.78%
